In [34]:
from bs4 import BeautifulSoup, Comment
import requests
import regex as re
import xml
from selenium import webdriver
import pandas as pd
import time

In [24]:
#USD

website = ["https://th.investing.com/currencies/usd-thb-historical-data",
"https://th.investing.com/currencies/jpy-thb-historical-data",
"https://th.investing.com/currencies/cny-thb-historical-data",
]


In [25]:
def remove_tag(html_str):
    TAG_RE = re.compile(r'<[^>]+>')
    text = TAG_RE.sub('!@', html_str)
    mylist = re.split(r'!@', text)
    clear_lst = []
    for clear_text in mylist:
        if (clear_text not in "\n") and (clear_text not in " ") and (clear_text not in ""):
            clear_lst.append(clear_text)
    return clear_lst
#https://www.guru99.com/python-regular-expressions-complete-tutorial.html

In [5]:
# def re_find(tag_start,tag_end,text):
#     a = re.search(tag_start, text)
#     if a == None:
#         return ''
#     text = text[a.start():]
#     b = re.search(tag_end, text)
#     text = text[:b.end()]
#     #print(a.start(),b.start())
#     return text

In [6]:
# def re_find_all(tag_start,tag_end,text):
#     list_all = []
#     while(True):
#         a = re.search(tag_start, text)
#         #print(a.start(),b.end())
#         if(a == None):
#             #print("break")
#             return list_all
#         text = text[a.start():]
#         b = re.search(tag_end, text)
#         list_all.append(text[:b.end()])
#         text=text[b.end():]
#     return list_all

#start browser

In [37]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
# executable_path param is not needed if you updated PATH

browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12564\924491811.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [38]:
browser.get(website[0])
time.sleep(3)
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")

In [8]:
#data = soup.find('div',{"id" : "results_box"}).find('table',{'class':'genTbl closedTbl historicalTbl'}).find('tbody').find_all('tr')

In [9]:
# data = re_find_all(r'<tr>',r'</tr>',re_find(r'<tbody>',r'</tbody>',re_find(r'<table class="genTbl closedTbl historicalTbl" id="curr_table" tablesorter="">',r'</table>',re_find(r'<div id="results_box">',r'</div>',str(soup)))))

In [53]:
data = re.findall('<tr>(.+?)</tr>',re.findall('<tbody>(.+?)</tbody>',re.findall('<table class="genTbl closedTbl historicalTbl" id="curr_table" tablesorter="">(.+?)</table>', re.findall('<div id="results_box">(.+?)</div>',re.sub(r'\n',"",str(soup)))[0])[0])[0])

#pack data to csv file

In [55]:
col = ['Date', 'Price', 'Open','High','Low','Change(%)']
df = pd.DataFrame()
df = df.reindex(columns = col)

In [56]:
for i,d in enumerate(data):
    dd = remove_tag(str(d))
    df.loc[len(df)] = dd

In [57]:
df

,Date,Price,Open,High,Low,Change(%)
0,"Apr 20, 2022",33.785,33.780,33.900,33.730,-0.01%
1,"Apr 19, 2022",33.790,33.690,33.810,33.650,0.30%
2,"Apr 18, 2022",33.690,33.630,33.720,33.580,0.24%
3,"Apr 15, 2022",33.610,33.690,33.720,33.600,-0.21%
4,"Apr 14, 2022",33.680,33.520,33.720,33.460,0.54%
5,"Apr 13, 2022",33.500,33.570,33.580,33.445,-0.24%
6,"Apr 12, 2022",33.580,33.610,33.710,33.460,-0.06%
7,"Apr 11, 2022",33.600,33.590,33.680,33.545,0.03%
8,"Apr 08, 2022",33.590,33.490,33.650,33.465,0.33%
9,"Apr 07, 2022",33.480,33.600,33.610,33.385,-0.30%


In [16]:
#df.to_csv('thb-usd.csv',index=False)
df.to_json('thb-usd.json')

#close browser

In [58]:
browser.quit()

In [13]:
f = open('text2.txt', 'w')
with open('text2.txt', 'w',encoding='utf-8') as f:
    f.write(str(soup))

In [113]:
#soup